In [2]:
import matplotlib.pyplot as plt
import numpy as np
from keras.datasets import mnist
from keras.models import Model, Sequential
from keras.layers import Input, Dense
from keras.utils import np_utils

#### Importar base de dados

In [3]:
(previsores_treinamento, classe_treinamento), (previsores_teste, classe_teste) = mnist.load_data()

#### Normalizar e dividir os dados

In [4]:
previsores_treinamento = previsores_treinamento.astype('float32') / 255
previsores_teste = previsores_teste.astype('float32') / 255

In [5]:
classe_dummy_treinamento = np_utils.to_categorical(classe_treinamento)
classe_dummy_teste = np_utils.to_categorical(classe_teste)

In [6]:
previsores_treinamento = previsores_treinamento.reshape((len(previsores_treinamento), np.prod(previsores_treinamento.shape[1:])))
previsores_teste = previsores_teste.reshape((len(previsores_teste), np.prod(previsores_teste.shape[1:])))

#### Estrutura do Autoencoder
- 784 - 32 - 784

In [7]:
fator_compactacao = 784 / 32
autoencoder = Sequential()
autoencoder.add(Dense(units = 32, activation = 'relu', input_dim = 784))
autoencoder.add(Dense(units = 784, activation = 'sigmoid'))
autoencoder.summary()
autoencoder.compile(optimizer = 'adam', loss = 'binary_crossentropy',
                    metrics = ['accuracy'])
autoencoder.fit(previsores_treinamento, previsores_treinamento,
                epochs = 100, batch_size = 256,
                validation_data = (previsores_teste, previsores_teste))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 32)                25120     
_________________________________________________________________
dense_2 (Dense)              (None, 784)               25872     
Total params: 50,992
Trainable params: 50,992
Non-trainable params: 0
_________________________________________________________________
Train on 60000 samples, validate on 10000 samples
Epoch 1/100
60000/60000 [==============================] - 4s 72us/step - loss: 0.2744 - accuracy: 0.7831 - val_loss: 0.1870 - val_accuracy: 0.7985
Epoch 2/100
60000/60000 [==============================] - 3s 50us/step - loss: 0.1680 - accuracy: 0.8030 - val_loss: 0.1507 - val_accuracy: 0.8051
Epoch 3/100
60000/60000 [==============================] - 3s 49us/step - loss: 0.1421 - accuracy: 0.8075 - val_loss: 0.1315 - val_accuracy: 0.8083
Epoch 4/100
60000/

60000/60000 [==============================] - 5s 76us/step - loss: 0.0926 - accuracy: 0.8142 - val_loss: 0.0916 - val_accuracy: 0.8133
Epoch 52/100
60000/60000 [==============================] - 4s 75us/step - loss: 0.0926 - accuracy: 0.8142 - val_loss: 0.0916 - val_accuracy: 0.8133
Epoch 53/100
60000/60000 [==============================] - 6s 93us/step - loss: 0.0926 - accuracy: 0.8142 - val_loss: 0.0916 - val_accuracy: 0.8133
Epoch 54/100
60000/60000 [==============================] - 5s 87us/step - loss: 0.0926 - accuracy: 0.8142 - val_loss: 0.0916 - val_accuracy: 0.8133
Epoch 55/100
60000/60000 [==============================] - 4s 74us/step - loss: 0.0926 - accuracy: 0.8142 - val_loss: 0.0915 - val_accuracy: 0.8133
Epoch 56/100
60000/60000 [==============================] - 4s 67us/step - loss: 0.0926 - accuracy: 0.8142 - val_loss: 0.0915 - val_accuracy: 0.8133
Epoch 57/100
60000/60000 [==============================] - 5s 80us/step - loss: 0.0926 - accuracy: 0.8142 - val_loss: 

#### Encoder

In [8]:
dimensao_original = Input(shape=(784,))
camada_encoder = autoencoder.layers[0]
encoder = Model(dimensao_original, camada_encoder(dimensao_original))


In [14]:
previsores_treinamento_codificados = encoder.predict(previsores_treinamento)
previsores_teste_codificados = encoder.predict(previsores_teste)

#### Modelo sem a redução de dimensionaldade
- (784 + 10) / 2

In [12]:
c1 = Sequential()
c1.add(Dense(units=397,activation = 'relu',input_dim = 784))
c1.add(Dense(units=397, activation = 'relu'))
c1.add(Dense(units = 10, activation = 'softmax'))
c1.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics=['accuracy'])
c1.fit(previsores_treinamento, classe_dummy_treinamento, batch_size=256, epochs = 100, validation_data = (previsores_teste, classe_dummy_teste))


Train on 60000 samples, validate on 10000 samples
Epoch 1/100
60000/60000 [==============================] - 25s 410us/step - loss: 0.2905 - accuracy: 0.9189 - val_loss: 0.1291 - val_accuracy: 0.9616
Epoch 2/100
60000/60000 [==============================] - 21s 348us/step - loss: 0.1015 - accuracy: 0.9694 - val_loss: 0.0844 - val_accuracy: 0.9735
Epoch 3/100
60000/60000 [==============================] - 15s 256us/step - loss: 0.0663 - accuracy: 0.9794 - val_loss: 0.0718 - val_accuracy: 0.9781
Epoch 4/100
60000/60000 [==============================] - 13s 220us/step - loss: 0.0431 - accuracy: 0.9871 - val_loss: 0.0640 - val_accuracy: 0.9805
Epoch 5/100
60000/60000 [==============================] - 12s 206us/step - loss: 0.0324 - accuracy: 0.9901 - val_loss: 0.0712 - val_accuracy: 0.9778
Epoch 6/100
60000/60000 [==============================] - 12s 205us/step - loss: 0.0249 - accuracy: 0.9919 - val_loss: 0.0746 - val_accuracy: 0.9802
Epoch 7/100
60000/60000 [=========================

60000/60000 [==============================] - 13s 219us/step - loss: 3.3107e-06 - accuracy: 1.0000 - val_loss: 0.1079 - val_accuracy: 0.9836
Epoch 55/100
60000/60000 [==============================] - 13s 216us/step - loss: 2.9408e-06 - accuracy: 1.0000 - val_loss: 0.1087 - val_accuracy: 0.9837
Epoch 56/100
60000/60000 [==============================] - 13s 215us/step - loss: 2.6208e-06 - accuracy: 1.0000 - val_loss: 0.1092 - val_accuracy: 0.9837
Epoch 57/100
60000/60000 [==============================] - 13s 216us/step - loss: 2.3302e-06 - accuracy: 1.0000 - val_loss: 0.1100 - val_accuracy: 0.9838
Epoch 58/100
60000/60000 [==============================] - 13s 210us/step - loss: 2.0697e-06 - accuracy: 1.0000 - val_loss: 0.1110 - val_accuracy: 0.9837
Epoch 59/100
60000/60000 [==============================] - 12s 207us/step - loss: 1.8461e-06 - accuracy: 1.0000 - val_loss: 0.1115 - val_accuracy: 0.9838
Epoch 60/100
60000/60000 [==============================] - 12s 205us/step - loss: 

#### Modelo com a redução de dimensionalidade

In [17]:
c2 = Sequential()
c2.add(Dense(units=21,activation = 'relu',input_dim = 32))
c2.add(Dense(units=21, activation = 'relu'))
c2.add(Dense(units = 10, activation = 'softmax'))
c2.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics=['accuracy'])
c2.fit(previsores_treinamento_codificados, classe_dummy_treinamento, batch_size=256, epochs = 1000, validation_data = (previsores_teste_codificados, classe_dummy_teste))


Train on 60000 samples, validate on 10000 samples
Epoch 1/1000
60000/60000 [==============================] - 1s 13us/step - loss: 2.1024 - accuracy: 0.3194 - val_loss: 1.2270 - val_accuracy: 0.5996
Epoch 2/1000
60000/60000 [==============================] - 1s 14us/step - loss: 0.8983 - accuracy: 0.7105 - val_loss: 0.6533 - val_accuracy: 0.7918
Epoch 3/1000
60000/60000 [==============================] - 1s 18us/step - loss: 0.5715 - accuracy: 0.8227 - val_loss: 0.4875 - val_accuracy: 0.8500
Epoch 4/1000
60000/60000 [==============================] - 1s 14us/step - loss: 0.4665 - accuracy: 0.8582 - val_loss: 0.4198 - val_accuracy: 0.8733
Epoch 5/1000
60000/60000 [==============================] - 1s 11us/step - loss: 0.4154 - accuracy: 0.8751 - val_loss: 0.3809 - val_accuracy: 0.8823
Epoch 6/1000
60000/60000 [==============================] - 1s 11us/step - loss: 0.3795 - accuracy: 0.8869 - val_loss: 0.3466 - val_accuracy: 0.8931
Epoch 7/1000
60000/60000 [==============================

60000/60000 [==============================] - 1s 11us/step - loss: 0.1783 - accuracy: 0.9454 - val_loss: 0.1790 - val_accuracy: 0.9448
Epoch 56/1000
60000/60000 [==============================] - 1s 12us/step - loss: 0.1777 - accuracy: 0.9463 - val_loss: 0.1719 - val_accuracy: 0.9500
Epoch 57/1000
60000/60000 [==============================] - 1s 11us/step - loss: 0.1750 - accuracy: 0.9471 - val_loss: 0.1700 - val_accuracy: 0.9484
Epoch 58/1000
60000/60000 [==============================] - 1s 9us/step - loss: 0.1760 - accuracy: 0.9465 - val_loss: 0.1730 - val_accuracy: 0.9484
Epoch 59/1000
60000/60000 [==============================] - 1s 9us/step - loss: 0.1736 - accuracy: 0.9480 - val_loss: 0.1657 - val_accuracy: 0.9487
Epoch 60/1000
60000/60000 [==============================] - 1s 10us/step - loss: 0.1737 - accuracy: 0.9477 - val_loss: 0.1665 - val_accuracy: 0.9494
Epoch 61/1000
60000/60000 [==============================] - 1s 9us/step - loss: 0.1740 - accuracy: 0.9469 - val_los

Epoch 110/1000
60000/60000 [==============================] - 1s 9us/step - loss: 0.1517 - accuracy: 0.9546 - val_loss: 0.1482 - val_accuracy: 0.9544
Epoch 111/1000
60000/60000 [==============================] - 1s 9us/step - loss: 0.1515 - accuracy: 0.9546 - val_loss: 0.1467 - val_accuracy: 0.9560
Epoch 112/1000
60000/60000 [==============================] - 1s 9us/step - loss: 0.1506 - accuracy: 0.9547 - val_loss: 0.1537 - val_accuracy: 0.9537
Epoch 113/1000
60000/60000 [==============================] - 1s 9us/step - loss: 0.1514 - accuracy: 0.9549 - val_loss: 0.1564 - val_accuracy: 0.9530
Epoch 114/1000
60000/60000 [==============================] - 1s 9us/step - loss: 0.1517 - accuracy: 0.9539 - val_loss: 0.1463 - val_accuracy: 0.9562
Epoch 115/1000
60000/60000 [==============================] - 1s 9us/step - loss: 0.1507 - accuracy: 0.9548 - val_loss: 0.1449 - val_accuracy: 0.9553
Epoch 116/1000
60000/60000 [==============================] - 1s 9us/step - loss: 0.1508 - accuracy:

60000/60000 [==============================] - 1s 9us/step - loss: 0.1400 - accuracy: 0.9579 - val_loss: 0.1437 - val_accuracy: 0.9559
Epoch 165/1000
60000/60000 [==============================] - 1s 10us/step - loss: 0.1397 - accuracy: 0.9579 - val_loss: 0.1392 - val_accuracy: 0.9571
Epoch 166/1000
60000/60000 [==============================] - 1s 9us/step - loss: 0.1392 - accuracy: 0.9579 - val_loss: 0.1451 - val_accuracy: 0.9554
Epoch 167/1000
60000/60000 [==============================] - 1s 9us/step - loss: 0.1394 - accuracy: 0.9580 - val_loss: 0.1426 - val_accuracy: 0.9576
Epoch 168/1000
60000/60000 [==============================] - 1s 10us/step - loss: 0.1391 - accuracy: 0.9582 - val_loss: 0.1429 - val_accuracy: 0.9577
Epoch 169/1000
60000/60000 [==============================] - 1s 9us/step - loss: 0.1401 - accuracy: 0.9582 - val_loss: 0.1408 - val_accuracy: 0.9580
Epoch 170/1000
60000/60000 [==============================] - 1s 10us/step - loss: 0.1383 - accuracy: 0.9577 - va

60000/60000 [==============================] - 1s 11us/step - loss: 0.1318 - accuracy: 0.9602 - val_loss: 0.1385 - val_accuracy: 0.9591
Epoch 219/1000
60000/60000 [==============================] - 1s 9us/step - loss: 0.1331 - accuracy: 0.9600 - val_loss: 0.1384 - val_accuracy: 0.9595
Epoch 220/1000
60000/60000 [==============================] - 1s 9us/step - loss: 0.1330 - accuracy: 0.9596 - val_loss: 0.1399 - val_accuracy: 0.9586
Epoch 221/1000
60000/60000 [==============================] - 1s 9us/step - loss: 0.1327 - accuracy: 0.9601 - val_loss: 0.1353 - val_accuracy: 0.9597
Epoch 222/1000
60000/60000 [==============================] - 1s 11us/step - loss: 0.1332 - accuracy: 0.9600 - val_loss: 0.1352 - val_accuracy: 0.9605
Epoch 223/1000
60000/60000 [==============================] - 1s 9us/step - loss: 0.1316 - accuracy: 0.9602 - val_loss: 0.1404 - val_accuracy: 0.9583
Epoch 224/1000
60000/60000 [==============================] - 1s 10us/step - loss: 0.1324 - accuracy: 0.9601 - va

...

60000/60000 [==============================] - 1s 9us/step - loss: 0.1110 - accuracy: 0.9660 - val_loss: 0.1369 - val_accuracy: 0.9605
Epoch 759/1000
60000/60000 [==============================] - 1s 9us/step - loss: 0.1113 - accuracy: 0.9656 - val_loss: 0.1395 - val_accuracy: 0.9592
Epoch 760/1000
60000/60000 [==============================] - 1s 9us/step - loss: 0.1108 - accuracy: 0.9663 - val_loss: 0.1324 - val_accuracy: 0.9608
Epoch 761/1000
60000/60000 [==============================] - 1s 9us/step - loss: 0.1112 - accuracy: 0.9651 - val_loss: 0.1293 - val_accuracy: 0.9628
Epoch 762/1000
60000/60000 [==============================] - 1s 9us/step - loss: 0.1115 - accuracy: 0.9657 - val_loss: 0.1323 - val_accuracy: 0.9623
Epoch 763/1000
60000/60000 [==============================] - 1s 9us/step - loss: 0.1115 - accuracy: 0.9656 - val_loss: 0.1324 - val_accuracy: 0.9619
Epoch 764/1000
60000/60000 [==============================] - 1s 9us/step - loss: 0.1114 - accuracy: 0.9661 - val_l

60000/60000 [==============================] - 1s 10us/step - loss: 0.1098 - accuracy: 0.9670 - val_loss: 0.1381 - val_accuracy: 0.9591
Epoch 813/1000
60000/60000 [==============================] - 1s 11us/step - loss: 0.1095 - accuracy: 0.9670 - val_loss: 0.1300 - val_accuracy: 0.9626
Epoch 814/1000
60000/60000 [==============================] - 1s 10us/step - loss: 0.1089 - accuracy: 0.9671 - val_loss: 0.1324 - val_accuracy: 0.9618
Epoch 815/1000
60000/60000 [==============================] - 1s 10us/step - loss: 0.1101 - accuracy: 0.9661 - val_loss: 0.1329 - val_accuracy: 0.9620
Epoch 816/1000
60000/60000 [==============================] - 1s 10us/step - loss: 0.1089 - accuracy: 0.9664 - val_loss: 0.1343 - val_accuracy: 0.9618
Epoch 817/1000
60000/60000 [==============================] - 1s 10us/step - loss: 0.1105 - accuracy: 0.9664 - val_loss: 0.1336 - val_accuracy: 0.9613
Epoch 818/1000
60000/60000 [==============================] - 1s 9us/step - loss: 0.1103 - accuracy: 0.9661 -

60000/60000 [==============================] - 1s 9us/step - loss: 0.1090 - accuracy: 0.9664 - val_loss: 0.1414 - val_accuracy: 0.9577
Epoch 867/1000
60000/60000 [==============================] - 1s 9us/step - loss: 0.1091 - accuracy: 0.9661 - val_loss: 0.1329 - val_accuracy: 0.9617
Epoch 868/1000
60000/60000 [==============================] - 1s 9us/step - loss: 0.1093 - accuracy: 0.9662 - val_loss: 0.1321 - val_accuracy: 0.9629
Epoch 869/1000
60000/60000 [==============================] - 1s 9us/step - loss: 0.1092 - accuracy: 0.9668 - val_loss: 0.1330 - val_accuracy: 0.9601
Epoch 870/1000
60000/60000 [==============================] - 1s 9us/step - loss: 0.1092 - accuracy: 0.9663 - val_loss: 0.1379 - val_accuracy: 0.9594
Epoch 871/1000
60000/60000 [==============================] - 1s 10us/step - loss: 0.1089 - accuracy: 0.9667 - val_loss: 0.1368 - val_accuracy: 0.9594
Epoch 872/1000
60000/60000 [==============================] - 1s 10us/step - loss: 0.1100 - accuracy: 0.9660 - val

60000/60000 [==============================] - 1s 9us/step - loss: 0.1096 - accuracy: 0.9662 - val_loss: 0.1318 - val_accuracy: 0.9631
Epoch 921/1000
60000/60000 [==============================] - 1s 11us/step - loss: 0.1081 - accuracy: 0.9668 - val_loss: 0.1411 - val_accuracy: 0.9589
Epoch 922/1000
60000/60000 [==============================] - 1s 9us/step - loss: 0.1086 - accuracy: 0.9668 - val_loss: 0.1303 - val_accuracy: 0.9625
Epoch 923/1000
60000/60000 [==============================] - 1s 9us/step - loss: 0.1081 - accuracy: 0.9671 - val_loss: 0.1423 - val_accuracy: 0.9598
Epoch 924/1000
60000/60000 [==============================] - 1s 9us/step - loss: 0.1090 - accuracy: 0.9667 - val_loss: 0.1339 - val_accuracy: 0.9617
Epoch 925/1000
60000/60000 [==============================] - 1s 9us/step - loss: 0.1080 - accuracy: 0.9666 - val_loss: 0.1336 - val_accuracy: 0.9613
Epoch 926/1000
60000/60000 [==============================] - 1s 9us/step - loss: 0.1083 - accuracy: 0.9668 - val_

60000/60000 [==============================] - 1s 12us/step - loss: 0.1080 - accuracy: 0.9670 - val_loss: 0.1368 - val_accuracy: 0.9603
Epoch 975/1000
60000/60000 [==============================] - 1s 12us/step - loss: 0.1073 - accuracy: 0.9673 - val_loss: 0.1354 - val_accuracy: 0.9604
Epoch 976/1000
60000/60000 [==============================] - 1s 11us/step - loss: 0.1077 - accuracy: 0.9671 - val_loss: 0.1320 - val_accuracy: 0.9625
Epoch 977/1000
60000/60000 [==============================] - 1s 12us/step - loss: 0.1081 - accuracy: 0.9668 - val_loss: 0.1300 - val_accuracy: 0.9632
Epoch 978/1000
60000/60000 [==============================] - 1s 10us/step - loss: 0.1078 - accuracy: 0.9670 - val_loss: 0.1367 - val_accuracy: 0.9606
Epoch 979/1000
60000/60000 [==============================] - 1s 10us/step - loss: 0.1076 - accuracy: 0.9672 - val_loss: 0.1318 - val_accuracy: 0.9620
Epoch 980/1000
60000/60000 [==============================] - 1s 10us/step - loss: 0.1084 - accuracy: 0.9664 

#### Acurácia dos Modelos:
- Sem redimensionamento: 98,44%.
    - Menos Veloz
- Com redimensionamento: 96,10%
    - Mais Veloz